In [13]:
import numpy as np
import pandas as pd
import random
from os.path import join
import os
import pickle
from glob import glob
import json
import sys

sys.path.append('../annotations/')
from color_harmony import extract_harmonies


In [9]:
DATA_DIR = '../data/'
df_scores_vectors = glob(DATA_DIR + 'scores_modifications*')
df_scores = pd.DataFrame()
for df_score in df_scores_vectors:
    df_score_l = pd.read_csv(df_score)
    df_scores = pd.concat([df_scores, df_score_l], axis=0)

df_scores = df_scores.sort_values('Final Score', ascending=False).reset_index()
df_scores.loc[df_scores['Variable'] == 'V1', 'Feature'] = 'V1'
df_scores.loc[df_scores['Variable'] == 'S1', 'Feature'] = 'S1'
df_scores['Method General'] = df_scores['Method'].str.split('_').apply(lambda x: x[0].replace('False', '').replace('True', ''))
print(df_scores['Method General'])
df_scores = df_scores.groupby(['Method General', 'Feature']).first()
df_scores = df_scores.reset_index()

modifications_vectors = glob(DATA_DIR + 'modifications*')
df_modifications_vectors = pd.DataFrame()
for df_modification in modifications_vectors:
    df_modification_l = pd.read_csv(df_modification)
    df_modifications_vectors = pd.concat([df_modifications_vectors, df_modification_l], axis=0)
print(df_modifications_vectors.columns)
df_modifications_vectors.loc[df_modifications_vectors['Variable'] == 'V1', 'Feature'] = 'V1'
df_modifications_vectors.loc[df_modifications_vectors['Variable'] == 'S1', 'Feature'] = 'S1'

separation_vectors = glob(DATA_DIR + '*_separation_vector*.csv')
df_sep_vecs = pd.DataFrame()
for df_sep_vec in separation_vectors:
    if 'modifications' not in df_sep_vec:
        df_sep_vec_l = pd.read_csv(df_sep_vec)
        df_sep_vecs = pd.concat([df_sep_vecs, df_sep_vec_l], axis=0)
df_sep_vecs.loc[df_sep_vecs['Variable'] == 'V1', 'Feature'] = 'V1'
df_sep_vecs.loc[df_sep_vecs['Variable'] == 'S1', 'Feature'] = 'S1'

df_scores = df_scores.merge(df_sep_vecs, left_on=['Feature','Variable','Method'], right_on=['Feature','Variable','Method'], how='left')
df_scores = df_scores.merge(df_modifications_vectors, left_on=['Feature','Variable','Method'], right_on=['Feature','Variable','Method'], how='left')
print(df_scores[['Feature', 'Variable', 'Method', 'Final Score']].sample(10), df_scores.shape)


/var/folders/s_/h4qqxs0s7mg2fw0j2hmf3jgh0000gn/T/ipykernel_78215/2156691675.py:6: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  df_scores = pd.concat([df_scores, df_score_l], axis=0)


0        ShapleyVec
1        ShapleyVec
2        ShapleyVec
3      InterfaceGAN
4        ShapleyVec
           ...     
215      StyleSpace
216      StyleSpace
217      StyleSpace
218      StyleSpace
219      StyleSpace
Name: Method General, Length: 220, dtype: object
Index(['seed', 'lambda', 'SSIM', 'H1', 'S1', 'V1', 'H2', 'S2', 'V2', 'H3',
       'S3', 'V3', 'H4', 'S4', 'V4', 'H5', 'S5', 'V5', 'Monochromatic',
       'Analogous', 'Complementary', 'Triadic', 'Split Complementary',
       'Double Complementary', 'Color', 'Variable', 'Feature', 'Space',
       'Method', 'Subfolder'],
      dtype='object')
      Feature Variable                             Method  Final Score
27470  Yellow    Color    ShapleyVec_True_0.25_8_Color_LR        0.967
41170  Yellow    Color    StyleSpaceTrue_20_8_Color_False        0.852
30317      BW    Color     StyleSpaceTrue_1_8_Color_False        0.405
10211     Red    Color     InterfaceGAN_LR_1_8_Color_True        0.879
15597  Yellow    Color  Interface

In [10]:
df_scores['Separation Vector (np)'] = [np.array([float(x.strip('[] ')) for x in row['Separation Vector'].replace('\n', ' ').split(' ') if x.strip('[] ') != '']) for i, row in df_scores.iterrows()]
df_scores['Separation Vector (np)']

0        [-0.0266842892, -0.00821213069, -0.00307416816...
1        [-0.0266842892, -0.00821213069, -0.00307416816...
2        [-0.0266842892, -0.00821213069, -0.00307416816...
3        [-0.0266842892, -0.00821213069, -0.00307416816...
4        [-0.0266842892, -0.00821213069, -0.00307416816...
                               ...                        
41595    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
41596    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
41597    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
41598    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
41599    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: Separation Vector (np), Length: 41600, dtype: object

<img src="../data/color_harmonies.jpg" width="300" height="500" style="align-text:center">

Checking which color harmony is obtained by which color modification

In [25]:
harmonies = {}
for method, group in df_scores[df_scores['lambda'] >= 10].groupby('Method General'):
    harmonies[method] = {}
    print('Assessing similarity among vectors computed with:', method)
    for feature, gg in group.groupby('Feature'):
        harmonies[method][feature] = []
        for hues in gg[['H1', 'H2', 'H3', 'H4', 'H5']].values:
            harmonies[method][feature].append(extract_harmonies(hues))
                

Assessing similarity among vectors computed with: InterfaceGAN
Assessing similarity among vectors computed with: ShapleyVec
Assessing similarity among vectors computed with: StyleSpace


in order:
- monochromatic
- analogous
- complementary
- triadic
- split complementary
- double complementary

In [26]:
for method in harmonies.keys():
    for feature in harmonies[method].keys():
        print(method, feature, list(np.round(np.mean(np.array(harmonies[method][feature]), axis=0), 3)))

InterfaceGAN BW [0.525, 0.878, 0.022, 0.0, 0.025, 0.008]
InterfaceGAN Blue [0.12, 0.268, 0.287, 0.003, 0.587, 0.148]
InterfaceGAN Brown [0.307, 0.93, 0.01, 0.0, 0.008, 0.002]
InterfaceGAN Cyan [0.133, 0.29, 0.36, 0.002, 0.485, 0.185]
InterfaceGAN Green [0.117, 0.38, 0.29, 0.003, 0.245, 0.14]
InterfaceGAN Magenta [0.143, 0.402, 0.093, 0.008, 0.143, 0.045]
InterfaceGAN Red [0.26, 0.523, 0.027, 0.0, 0.038, 0.012]
InterfaceGAN S1 [0.212, 0.798, 0.01, 0.0, 0.032, 0.008]
InterfaceGAN V1 [0.187, 0.63, 0.138, 0.002, 0.148, 0.075]
InterfaceGAN Yellow [0.258, 0.842, 0.022, 0.0, 0.02, 0.02]
ShapleyVec BW [0.518, 0.847, 0.022, 0.0, 0.018, 0.015]
ShapleyVec Blue [0.025, 0.137, 0.373, 0.005, 0.612, 0.192]
ShapleyVec Brown [0.048, 0.863, 0.02, 0.0, 0.032, 0.01]
ShapleyVec Cyan [0.055, 0.177, 0.373, 0.0, 0.61, 0.217]
ShapleyVec Green [0.018, 0.177, 0.337, 0.008, 0.325, 0.125]
ShapleyVec Magenta [0.085, 0.28, 0.105, 0.005, 0.117, 0.06]
ShapleyVec Red [0.04, 0.14, 0.087, 0.005, 0.153, 0.052]
ShapleyVec 

Which colors create the most structural change and going into which color